# Example Predictor: Linear Rollout Predictor

This example contains basic functionality for training and evaluating a linear predictor that rolls out predictions day-by-day.

First, a training data set is created from historical case and npi data.

Second, a linear model is trained to predict future cases from prior case data along with prior and future npi data.
The model is an off-the-shelf sklearn Lasso model, that uses a positive weight constraint to enforce the assumption that increased npis has a negative correlation with future cases.

Third, a sample evaluation set is created, and the predictor is applied to this evaluation set to produce prediction results in the correct format.

## Training

In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

### Copy the data locally

In [2]:
# Main source for the training data
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
# Local file
DATA_FILE = 'data/OxCGRT_latest.csv'

In [3]:
import os
import urllib.request
if not os.path.exists('data'):
    os.mkdir('data')
urllib.request.urlretrieve(DATA_URL, DATA_FILE)

('data/OxCGRT_latest.csv', <http.client.HTTPMessage at 0x7f1f0ceb1b70>)

In [4]:
# Load historical data from local file
df = pd.read_csv(DATA_FILE, 
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)

In [5]:
df.columns

Index(['CountryName', 'CountryCode', 'RegionName', 'RegionCode',
       'Jurisdiction', 'Date', 'C1_School closing', 'C1_Flag',
       'C2_Workplace closing', 'C2_Flag', 'C3_Cancel public events', 'C3_Flag',
       'C4_Restrictions on gatherings', 'C4_Flag', 'C5_Close public transport',
       'C5_Flag', 'C6_Stay at home requirements', 'C6_Flag',
       'C7_Restrictions on internal movement', 'C7_Flag',
       'C8_International travel controls', 'E1_Income support', 'E1_Flag',
       'E2_Debt/contract relief', 'E3_Fiscal measures',
       'E4_International support', 'H1_Public information campaigns',
       'H1_Flag', 'H2_Testing policy', 'H3_Contact tracing',
       'H4_Emergency investment in healthcare', 'H5_Investment in vaccines',
       'H6_Facial Coverings', 'H6_Flag', 'M1_Wildcard', 'ConfirmedCases',
       'ConfirmedDeaths', 'StringencyIndex', 'StringencyIndexForDisplay',
       'StringencyLegacyIndex', 'StringencyLegacyIndexForDisplay',
       'GovernmentResponseIndex', 'Gove

In [6]:
# For testing, restrict training data to that before a hypothetical predictor submission date
HYPOTHETICAL_SUBMISSION_DATE = np.datetime64("2020-07-31")
df = df[df.Date <= HYPOTHETICAL_SUBMISSION_DATE]

In [7]:
# Add RegionID column that combines CountryName and RegionName for easier manipulation of data
df['GeoID'] = df['CountryName'] + '__' + df['RegionName'].astype(str)

In [8]:
# Add new cases column
df['NewCases'] = df.groupby('GeoID').ConfirmedCases.diff().fillna(0)

In [9]:
# Keep only columns of interest
id_cols = ['CountryName',
           'RegionName',
           'GeoID',
           'Date']
cases_col = ['NewCases']
npi_cols = ['C1_School closing',
            'C2_Workplace closing',
            'C3_Cancel public events',
            'C4_Restrictions on gatherings',
            'C5_Close public transport',
            'C6_Stay at home requirements',
            'C7_Restrictions on internal movement',
            'C8_International travel controls',
            'H1_Public information campaigns',
            'H2_Testing policy',
            'H3_Contact tracing',
            'H6_Facial Coverings']
df = df[id_cols + cases_col + npi_cols]

In [10]:
# Fill any missing case values by interpolation and setting NaNs to 0
df.update(df.groupby('GeoID').NewCases.apply(
    lambda group: group.interpolate()).fillna(0))

In [11]:
# Fill any missing NPIs by assuming they are the same as previous day
for npi_col in npi_cols:
    df.update(df.groupby('GeoID')[npi_col].ffill().fillna(0))

In [12]:
df

,CountryName,RegionName,GeoID,Date,NewCases,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings
0,Aruba,NaN,Aruba__nan,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,NaN,Aruba__nan,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aruba,NaN,Aruba__nan,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aruba,NaN,Aruba__nan,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aruba,NaN,Aruba__nan,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87064,Zimbabwe,NaN,Zimbabwe__nan,2020-07-27,78.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
87065,Zimbabwe,NaN,Zimbabwe__nan,2020-07-28,192.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
87066,Zimbabwe,NaN,Zimbabwe__nan,2020-07-29,113.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
87067,Zimbabwe,NaN,Zimbabwe__nan,2020-07-30,62.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0


In [13]:
df['CountryName'].unique()

array(['Aruba', 'Afghanistan', 'Angola', 'Albania', 'Andorra',
       'United Arab Emirates', 'Argentina', 'Australia', 'Austria',
       'Azerbaijan', 'Burundi', 'Belgium', 'Benin', 'Burkina Faso',
       'Bangladesh', 'Bulgaria', 'Bahrain', 'Bahamas',
       'Bosnia and Herzegovina', 'Belarus', 'Belize', 'Bermuda',
       'Bolivia', 'Brazil', 'Barbados', 'Brunei', 'Bhutan', 'Botswana',
       'Central African Republic', 'Canada', 'Switzerland', 'Chile',
       'China', "Cote d'Ivoire", 'Cameroon',
       'Democratic Republic of Congo', 'Congo', 'Colombia', 'Comoros',
       'Cape Verde', 'Costa Rica', 'Cuba', 'Cyprus', 'Czech Republic',
       'Germany', 'Djibouti', 'Dominica', 'Denmark', 'Dominican Republic',
       'Algeria', 'Ecuador', 'Egypt', 'Eritrea', 'Spain', 'Estonia',
       'Ethiopia', 'Finland', 'Fiji', 'France', 'Faeroe Islands', 'Gabon',
       'United Kingdom', 'Georgia', 'Ghana', 'Guinea', 'Gambia', 'Greece',
       'Greenland', 'Guatemala', 'Guam', 'Guyana', 'Hong Ko

In [14]:
# Set number of past days to use to make predictions
nb_lookback_days = 30

# Create training data across all countries for predicting one day ahead
X_cols = cases_col + npi_cols
y_col = cases_col
X_samples = []
y_samples = []
geo_ids = df.GeoID.unique()
for g in geo_ids:
    gdf = df[df.GeoID == g]
    all_case_data = np.array(gdf[cases_col])
    all_npi_data = np.array(gdf[npi_cols])

    # Create one sample for each day where we have enough data
    # Each sample consists of cases and npis for previous nb_lookback_days
    nb_total_days = len(gdf)
    for d in range(nb_lookback_days, nb_total_days - 1):
        X_cases = all_case_data[d-nb_lookback_days:d]

        # Take negative of npis to support positive
        # weight constraint in Lasso.
        X_npis = -all_npi_data[d - nb_lookback_days:d]

        # Flatten all input data so it fits Lasso input format.
        X_sample = np.concatenate([X_cases.flatten(),
                                   X_npis.flatten()])
        y_sample = all_case_data[d + 1]
        X_samples.append(X_sample)
        y_samples.append(y_sample)

X_samples = np.array(X_samples)
y_samples = np.array(y_samples).flatten()

In [15]:
# Helpful function to compute mae
def mae(pred, true):
    return np.mean(np.abs(pred - true))

In [16]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_samples,
                                                    y_samples,
                                                    test_size=0.2,
                                                    random_state=301)

In [17]:
# # Create and train Lasso model.
# # Set positive=True to enforce assumption that cases are positively correlated
# # with future cases and npis are negatively correlated.
# model = Lasso(alpha=0.1,
#               precompute=True,
#               max_iter=10000,
#               positive=True,
#               selection='random')
# # Fit model
# model.fit(X_train, y_train)

In [18]:
# from sklearn.linear_model import Ridge
# model = Ridge(alpha = 0.5)
# # Fit model
# model.fit(X_train, y_train)

In [19]:
# from sklearn.experimental import enable_hist_gradient_boosting
# from sklearn.ensemble import HistGradientBoostingRegressor
# model = HistGradientBoostingRegressor(loss='least_squares',  learning_rate=0.1, max_iter=1000, max_leaf_nodes=31, max_depth=5, min_samples_leaf=20, l2_regularization=0.5, max_bins=255, monotonic_cst=None, warm_start=False, early_stopping='auto', scoring='loss', validation_fraction=0.1, n_iter_no_change=10, tol=1e-07, verbose=0, random_state=None)
# model.fit(X_train, y_train)

In [20]:
# from sklearn.ensemble import GradientBoostingRegressor
# model = GradientBoostingRegressor()
# model.fit(X_train, y_train)

In [21]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
model = AdaBoostRegressor(base_estimator = DecisionTreeRegressor(max_depth=10), n_estimators = 500, loss = 'square')
model.fit(X_train, y_train)

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=10),
                  loss='square', n_estimators=500)

In [22]:
# Evaluate model
train_preds = model.predict(X_train)
train_preds = np.maximum(train_preds, 0) # Don't predict negative cases
print('Train MAE:', mae(train_preds, y_train))

test_preds = model.predict(X_test)
test_preds = np.maximum(test_preds, 0) # Don't predict negative cases
print('Test MAE:', mae(test_preds, y_test))

Train MAE: 231.2096294135742
Test MAE: 283.4694259694012


In [23]:
# # Inspect the learned feature coefficients for the model
# # to see what features it's paying attention to.

# # Give names to the features
# x_col_names = []
# for d in range(-nb_lookback_days, 0):
#     x_col_names.append('Day ' + str(d) + ' ' + cases_col[0])
# for d in range(-nb_lookback_days, 1):
#     for col_name in npi_cols:
#         x_col_names.append('Day ' + str(d) + ' ' + col_name)

# # View non-zero coefficients
# for (col, coeff) in zip(x_col_names, list(model.coef_)):
#     if coeff != 0.:
#         print(col, coeff)
# print('Intercept', model.intercept_)

In [24]:
# Save model to file
if not os.path.exists('models'):
    os.mkdir('models')
with open('models/model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

## Evaluation

Now that the predictor has been trained and saved, this section contains the functionality for evaluating it on sample evaluation data.

In [25]:
# Reload the module to get the latest changes
import predict
from importlib import reload
reload(predict)
from predict import predict_df

In [26]:
list_countries = sorted(list(set(df.CountryName)))
hist_ips_df = pd.read_csv("data/2020-09-30_historical_ip.csv",
                              parse_dates=['Date'],
                              encoding="ISO-8859-1",
                              dtype={"RegionName": str},
                              error_bad_lines=True)
hist_ips_df = hist_ips_df[hist_ips_df.CountryName.isin(list_countries)]
hist_ips_df.to_csv("data/2020-09-30_historical_ip_new.csv" , index = False) 

In [27]:
%%time
preds_df = predict_df("2020-08-01", "2020-08-31", path_to_ips_file="data/2020-09-30_historical_ip_new.csv", verbose=True)


Predicting for Aruba__nan
2020-08-01: 265.9029850746269
2020-08-02: 293.048309178744
2020-08-03: 298.4912099276112
2020-08-04: 314.4155844155844
2020-08-05: 367.6505248899424
2020-08-06: 401.5450669914738
2020-08-07: 457.35806074766356
2020-08-08: 507.6719160104987
2020-08-09: 594.1411764705882
2020-08-10: 645.2005649717514
2020-08-11: 723.9504504504505
2020-08-12: 778.104
2020-08-13: 794.875
2020-08-14: 827.6923076923077
2020-08-15: 905.552
2020-08-16: 1047.929411764706
2020-08-17: 1143.2547444007523
2020-08-18: 1230.332110091743
2020-08-19: 1366.3333333333333
2020-08-20: 1165.3647058823528
2020-08-21: 1302.2609939220592
2020-08-22: 1319.2734067663257
2020-08-23: 1387.9222972972973
2020-08-24: 1402.2289669861555
2020-08-25: 1437.4609540636043
2020-08-26: 1425.7229437229437
2020-08-27: 1410.2761780104713
2020-08-28: 1416.5485786254048
2020-08-29: 1620.71758135017
2020-08-30: 1714.21875
2020-08-31: 1827.0

Predicting for Afghanistan__nan
2020-08-01: 403.5917602996255
2020-08-02: 437.03

2020-08-01: 2474.945054945055
2020-08-02: 2520.5945171849426
2020-08-03: 2235.1178144564597
2020-08-04: 2529.43358568479
2020-08-05: 2352.549465240642
2020-08-06: 2184.2649842271294
2020-08-07: 2123.1085582998276
2020-08-08: 2266.226804123711
2020-08-09: 2088.4631994427677
2020-08-10: 1978.2153008769278
2020-08-11: 1733.242673992674
2020-08-12: 1892.0616883116884
2020-08-13: 1849.471766848816
2020-08-14: 1678.5689655172414
2020-08-15: 1673.8011424491253
2020-08-16: 1678.791856348002
2020-08-17: 1561.3857142857144
2020-08-18: 1567.7771084337348
2020-08-19: 1561.3857142857144
2020-08-20: 1530.1283607979185
2020-08-21: 1486.7685950413222
2020-08-22: 1486.7685950413222
2020-08-23: 1523.221237268955
2020-08-24: 1478.9318181818182
2020-08-25: 1496.951048951049
2020-08-26: 1451.844589409056
2020-08-27: 1451.844589409056
2020-08-28: 1437.4609540636043
2020-08-29: 1437.0379746835442
2020-08-30: 1450.3586678052945
2020-08-31: 1435.0263157894738

Predicting for Burundi__nan
2020-08-01: 229.790130

2020-08-18: 1572.413208389112
2020-08-19: 1406.6343669250646
2020-08-20: 1273.9642857142858
2020-08-21: 1416.5485786254048
2020-08-22: 1486.7685950413222
2020-08-23: 1552.391304347826
2020-08-24: 1558.875
2020-08-25: 1590.6363636363637
2020-08-26: 1478.429411764706
2020-08-27: 1541.0182128628344
2020-08-28: 1435.0263157894738
2020-08-29: 1545.6255773672056
2020-08-30: 1570.011320754717
2020-08-31: 1558.875

Predicting for Belarus__nan
2020-08-01: 1437.4609540636043
2020-08-02: 1558.875
2020-08-03: 1570.011320754717
2020-08-04: 1611.5749324063345
2020-08-05: 1572.413208389112
2020-08-06: 1560.2747138397503
2020-08-07: 1561.3857142857144
2020-08-08: 1643.6465280502157
2020-08-09: 1612.8761904761905
2020-08-10: 1612.8761904761905
2020-08-11: 1572.413208389112
2020-08-12: 1612.8761904761905
2020-08-13: 1572.413208389112
2020-08-14: 1612.8761904761905
2020-08-15: 1570.011320754717
2020-08-16: 1572.413208389112
2020-08-17: 1574.2950495049504
2020-08-18: 1574.2950495049504
2020-08-19: 1590.63

2020-08-20: 1550.1631504922643
2020-08-21: 1724.7342792647532
2020-08-22: 1685.0296096904442
2020-08-23: 1617.0939759036144
2020-08-24: 1795.0804416403785
2020-08-25: 1620.71758135017
2020-08-26: 1617.0939759036144
2020-08-27: 1654.1842105263158
2020-08-28: 1599.9110225763611
2020-08-29: 1748.8464730290457
2020-08-30: 1766.8474576271187
2020-08-31: 1744.2295514511873

Predicting for Central African Republic__nan
2020-08-01: 243.8987341772152
2020-08-02: 243.8987341772152
2020-08-03: 213.1256038647343
2020-08-04: 213.58227848101265
2020-08-05: 260.62068965517244
2020-08-06: 333.82978723404256
2020-08-07: 399.0625
2020-08-08: 448.75
2020-08-09: 483.3636363636364
2020-08-10: 588.762979683973
2020-08-11: 748.5644796380091
2020-08-12: 804.7474747474747
2020-08-13: 887.1666666666666
2020-08-14: 871.9005235602094
2020-08-15: 1060.4
2020-08-16: 1148.043659043659
2020-08-17: 1293.25
2020-08-18: 1601.3225806451612
2020-08-19: 1626.875
2020-08-20: 1503.4333333333334
2020-08-21: 1468.016393442623


2020-08-15: 1448.8992137240887
2020-08-16: 1547.8177842565597
2020-08-17: 1781.7837837837837
2020-08-18: 1669.5294117647059
2020-08-19: 1805.6119402985075
2020-08-20: 1806.8677685950413
2020-08-21: 2019.8743994509266
2020-08-22: 1806.8677685950413
2020-08-23: 1939.278431372549
2020-08-24: 1877.8504847564536
2020-08-25: 1951.241134751773
2020-08-26: 1768.9919162470183
2020-08-27: 1878.5364238410596
2020-08-28: 1816.2610441767067
2020-08-29: 1854.7531914893616
2020-08-30: 1878.5364238410596
2020-08-31: 1865.581818181818

Predicting for Colombia__nan
2020-08-01: 7355.0
2020-08-02: 7433.987369985141
2020-08-03: 7419.8
2020-08-04: 7571.1
2020-08-05: 6922.034782608695
2020-08-06: 7169.133891213389
2020-08-07: 7145.104895104895
2020-08-08: 7424.5
2020-08-09: 7535.6
2020-08-10: 7334.231578947369
2020-08-11: 7597.181818181818
2020-08-12: 7540.15243902439
2020-08-13: 7617.2962962962965
2020-08-14: 7720.714285714285
2020-08-15: 7841.178571428572
2020-08-16: 7932.035714285715
2020-08-17: 7979.8509

...

2020-08-18: 4601.42205323194
2020-08-19: 5040.430555555556
2020-08-20: 5082.01253298153
2020-08-21: 4441.0
2020-08-22: 3927.487870619946
2020-08-23: 3499.104278074866
2020-08-24: 3525.2
2020-08-25: 3520.0666666666666
2020-08-26: 3405.325
2020-08-27: 3328.292650918635
2020-08-28: 3101.676646706587
2020-08-29: 2957.293103448276
2020-08-30: 2936.655109489051
2020-08-31: 2769.0038167938933

Predicting for United States__Minnesota
2020-08-01: 5800.772241992882
2020-08-02: 6260.823529411765
2020-08-03: 7056.897435897436
2020-08-04: 6400.28125
2020-08-05: 5768.0109375
2020-08-06: 4307.225806451613
2020-08-07: 4596.422480620155
2020-08-08: 5766.169421487603
2020-08-09: 5858.476190476191
2020-08-10: 5661.225609756098
2020-08-11: 4568.58275862069
2020-08-12: 3927.487870619946
2020-08-13: 3826.4655172413795
2020-08-14: 3865.8348017621147
2020-08-15: 3679.4054266614235
2020-08-16: 3555.8947368421054
2020-08-17: 3430.976389946687
2020-08-18: 3230.7475421348313
2020-08-19: 2900.2608695652175
2020-08

2020-08-02: 3381.3270735524256
2020-08-03: 3745.25641025641
2020-08-04: 3111.0
2020-08-05: 2858.1608174770968
2020-08-06: 2556.0416666666665
2020-08-07: 2900.2608695652175
2020-08-08: 2647.0
2020-08-09: 2392.1824587706146
2020-08-10: 2379.96875
2020-08-11: 2096.0942028985505
2020-08-12: 1993.4755244755245
2020-08-13: 1930.0982800982802
2020-08-14: 1942.1821184995738
2020-08-15: 1766.8474576271187
2020-08-16: 1729.2483108108108
2020-08-17: 1670.7442326980943
2020-08-18: 1773.0
2020-08-19: 1678.5689655172414
2020-08-20: 1611.5749324063345
2020-08-21: 1670.7442326980943
2020-08-22: 1611.5749324063345
2020-08-23: 1612.8761904761905
2020-08-24: 1561.3857142857144
2020-08-25: 1727.1478743068392
2020-08-26: 1620.71758135017
2020-08-27: 1486.7685950413222
2020-08-28: 1268.4164244186047
2020-08-29: 1478.9318181818182
2020-08-30: 1478.429411764706
2020-08-31: 1485.0957484816006

Predicting for United States__New Mexico
2020-08-01: 2088.4631994427677
2020-08-02: 1877.8504847564536
2020-08-03: 200

2020-08-24: 1545.6255773672056
2020-08-25: 1417.9252747252747
2020-08-26: 1381.421768707483
2020-08-27: 1612.8761904761905
2020-08-28: 1402.2289669861555
2020-08-29: 1570.011320754717
2020-08-30: 1636.8257575757575
2020-08-31: 1558.875

Predicting for United States__South Carolina
2020-08-01: 1611.5749324063345
2020-08-02: 1466.4910714285713
2020-08-03: 1623.984198645598
2020-08-04: 1466.0714285714287
2020-08-05: 1356.5699481865286
2020-08-06: 1161.4784688995214
2020-08-07: 1623.984198645598
2020-08-08: 1515.8462039752362
2020-08-09: 1486.7685950413222
2020-08-10: 1636.8257575757575
2020-08-11: 1474.8083750568958
2020-08-12: 1383.9178082191781
2020-08-13: 1435.0263157894738
2020-08-14: 1481.571814030718
2020-08-15: 1486.7685950413222
2020-08-16: 1486.7685950413222
2020-08-17: 1481.571814030718
2020-08-18: 1612.8761904761905
2020-08-19: 1451.844589409056
2020-08-20: 1561.3519206939282
2020-08-21: 1619.025641025641
2020-08-22: 1523.221237268955
2020-08-23: 1486.7685950413222
2020-08-24: 

2020-08-13: 1651.9433691756271
2020-08-14: 1612.8761904761905
2020-08-15: 1361.4
2020-08-16: 1694.611111111111
2020-08-17: 1647.1601272534465
2020-08-18: 1682.4923076923078
2020-08-19: 1620.71758135017
2020-08-20: 1643.6465280502157
2020-08-21: 1523.221237268955
2020-08-22: 1613.7877094972066
2020-08-23: 1886.910447761194
2020-08-24: 1620.71758135017
2020-08-25: 1725.1510710259302
2020-08-26: 1416.5485786254048
2020-08-27: 1497.016584402764
2020-08-28: 1383.9178082191781
2020-08-29: 1468.016393442623
2020-08-30: 1560.2747138397503
2020-08-31: 1481.571814030718

Predicting for United States__Wisconsin
2020-08-01: 6372.868965517241
2020-08-02: 6505.959183673469
2020-08-03: 5601.4
2020-08-04: 4312.925
2020-08-05: 3527.497191011236
2020-08-06: 3055.693544271498
2020-08-07: 5396.110655737705
2020-08-08: 4378.663934426229
2020-08-09: 3923.03125
2020-08-10: 3231.176470588235
2020-08-11: 3001.034327009937
2020-08-12: 2900.2608695652175
2020-08-13: 3393.1
2020-08-14: 2880.1666666666665
2020-08-

2020-08-04: 211.50259067357513
2020-08-05: 292.58125
2020-08-06: 303.44444444444446
2020-08-07: 347.39716312056737
2020-08-08: 452.5079365079365
2020-08-09: 549.8716577540107
2020-08-10: 792.0926517571885
2020-08-11: 862.3356643356643
2020-08-12: 1042.0
2020-08-13: 1283.0
2020-08-14: 1449.4204793028323
2020-08-15: 1593.341062079282
2020-08-16: 1936.5
2020-08-17: 1967.7153846153847
2020-08-18: 1951.241134751773
2020-08-19: 1955.0881057268723
2020-08-20: 2019.8743994509266
2020-08-21: 2046.0166139240507
2020-08-22: 2204.6971514242878
2020-08-23: 2196.608938547486
2020-08-24: 2053.9321333009
2020-08-25: 2018.8854166666667
2020-08-26: 2035.2375478927204
2020-08-27: 2132.0071856287427
2020-08-28: 2075.471193415638
2020-08-29: 2160.7928759894457
2020-08-30: 2063.4003813155387
2020-08-31: 2065.3389830508477

Predicting for South Africa__nan
2020-08-01: 2229.0885780885783
2020-08-02: 2053.9321333009
2020-08-03: 2123.1085582998276
2020-08-04: 2043.4161073825503
2020-08-05: 1865.581818181818
202

In [28]:
# Check the predictions
preds_df.head()

,CountryName,RegionName,Date,PredictedDailyNewCases
213,Aruba,NaN,2020-08-01,265.902985
214,Aruba,NaN,2020-08-02,293.048309
215,Aruba,NaN,2020-08-03,298.491210
216,Aruba,NaN,2020-08-04,314.415584
217,Aruba,NaN,2020-08-05,367.650525


# Validation
This is how the predictor is going to be called during the competition.  
!!! PLEASE DO NOT CHANGE THE API !!!

In [29]:
!python3 predict.py -s 2020-08-01 -e 2020-08-04 -ip data/2020-09-30_historical_ip_new.csv -o predictions/2020-08-01_2020-08-04.csv

Generating predictions from 2020-08-01 to 2020-08-04...
Saved predictions to predictions/2020-08-01_2020-08-04.csv
Done!


In [30]:
!head predictions/2020-08-01_2020-08-04.csv

CountryName,RegionName,Date,PredictedDailyNewCases
Aruba,,2020-08-01,265.9029850746269
Aruba,,2020-08-02,293.048309178744
Aruba,,2020-08-03,298.4912099276112
Aruba,,2020-08-04,314.4155844155844
Afghanistan,,2020-08-01,403.5917602996255
Afghanistan,,2020-08-02,437.03816046966733
Afghanistan,,2020-08-03,455.265306122449
Afghanistan,,2020-08-04,510.7369246861925
Angola,,2020-08-01,388.89102564102564


# Test cases
We can generate a prediction file. Let's validate a few cases...

In [31]:
import os
from predictor_validation import validate_submission

def validate(start_date, end_date, ip_file, output_file):
    # First, delete any potential old file
    try:
        os.remove(output_file)
    except OSError:
        pass
    
    # Then generate the prediction, calling the official API
    !python3 predict.py -s {start_date} -e {end_date} -ip {ip_file} -o {output_file}
    
    # And validate it
    errors = validate_submission(start_date, end_date, ip_file, output_file)
    if errors:
        for error in errors:
            print(error)
    else:
        print("All good!")

## 4 days, no gap
- All countries and regions
- Official number of cases is known up to start_date
- Intervention Plans are the official ones

In [ ]:
# validate(start_date="2020-08-01",
#          end_date="2020-08-04",
#          ip_file="data/2020-09-30_historical_ip_new.csv",
#          output_file="predictions/val_4_days.csv")

validate(start_date="2020-08-01",
         end_date="2020-08-31",
         ip_file="data/2020-09-30_historical_ip_new.csv",
         output_file="predictions/val_august.csv")

Generating predictions from 2020-08-01 to 2020-08-31...


## 1 month in the future
- 2 countries only
- there's a gap between date of last known number of cases and start_date
- For future dates, Intervention Plans contains scenarios for which predictions are requested to answer the question: what will happen if we apply these plans?

In [ ]:
# %%time
# validate(start_date="2021-01-01",
#          end_date="2021-01-31",
#          ip_file="validation/data/future_ip.csv",
#          output_file="predictions/val_1_month_future.csv")

## 180 days, from a future date, all countries and regions
- Prediction start date is 1 week from now. (i.e. assuming submission date is 1 week from now)  
- Prediction end date is 6 months after start date.  
- Prediction is requested for all available countries and regions.  
- Intervention plan scenario: freeze last known intervention plans for each country and region.  

As the number of cases is not known yet between today and start date, but the model relies on them, the model has to predict them in order to use them.  
This test is the most demanding test. It should take less than 1 hour to generate the prediction file.

### Generate the scenario

In [ ]:
# from datetime import datetime, timedelta

# start_date = datetime.now() + timedelta(days=7)
# start_date_str = start_date.strftime('%Y-%m-%d')
# end_date = start_date + timedelta(days=180)
# end_date_str = end_date.strftime('%Y-%m-%d')
# print(f"Start date: {start_date_str}")
# print(f"End date: {end_date_str}")

In [ ]:
# from validation.scenario_generator import get_raw_data, generate_scenario, NPI_COLUMNS
# DATA_FILE = 'data/OxCGRT_latest.csv'
# latest_df = get_raw_data(DATA_FILE, latest=True)
# scenario_df = generate_scenario(start_date_str, end_date_str, latest_df, countries=None, scenario="Freeze")
# scenario_file = "predictions/180_days_future_scenario.csv"
# scenario_df.to_csv(scenario_file, index=False)
# print(f"Saved scenario to {scenario_file}")

### Check it

In [ ]:
# %%time
# validate(start_date=start_date_str,
#          end_date=end_date_str,
#          ip_file=scenario_file,
#          output_file="predictions/val_6_month_future.csv")

## SPAIN

In [ ]:
# start_date = datetime.now() + timedelta(days=7)
# start_date_str = start_date.strftime('%Y-%m-%d')
# end_date = start_date + timedelta(days=180)
# end_date_str = end_date.strftime('%Y-%m-%d')
# print(f"Start date: {start_date_str}")
# print(f"End date: {end_date_str}")

# DATA_FILE = 'data/OxCGRT_latest.csv'
# latest_df = get_raw_data(DATA_FILE, latest=True)
# scenario_df = generate_scenario(start_date_str, end_date_str, latest_df, countries=['France'], scenario="Freeze")
# scenario_file = "predictions/180_days_future_scenario_france_freeze.csv"
# scenario_df.to_csv(scenario_file, index=False)
# print(f"Saved scenario to {scenario_file}")

# validate(start_date=start_date_str,
#          end_date=end_date_str,
#          ip_file=scenario_file,
#          output_file="predictions/val_6_month_future_france_freeze.csv")

In [ ]:
# df_verify = pd.read_csv('predictions/val_6_month_future_spain_max.csv')
# x = df_verify['Date'].to_numpy()
# y = df_verify['PredictedDailyNewCases'].to_numpy()

# %matplotlib inline
# import matplotlib.pyplot as plt
# plt.plot(x,y)